### Train/Dev/Test sets
Dev and Test sets should contain the most recent data.

TODO: 
- Concatenate all data from 2014 and feed them to prep_for_ml
- fix read_metar.

In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

from utils import geoutils
from utils import datautils

pd.options.mode.chained_assignment = None

/home/atkm/code/nyc-taxi/venv/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
# takes a raw (an output of read_rides) DataFrame, and counts the number of rides in each grid cell.
def counts_by_grid_cell(df):
    df = datautils.clean_rides(df)
    df = datautils.add_grid_cols(df)
    count = df.groupby(['grid_x', 'grid_y']).size()
    return count

In [2]:
rides = datautils.read_rides('data/yellow_tripdata_2014-01_small.csv')
metar = datautils.read_metar('data/metar_data/lga_2014-01.csv')
X, y = datautils.prep_for_ml(rides, metar)

In [3]:
# Note X.values, y.values
X_train, X_test, y_train, y_test = train_test_split(X.values, y.values, test_size=0.1, random_state=1)
X_train, X_dev, y_train, y_dev = train_test_split(X_train, y_train,test_size=0.1, random_state=2)

In [4]:
rf = RandomForestRegressor()
rf.fit(X_train, y_train)
rf.score(X_dev, y_dev)

0.46052004461968754

## Exploratory work
Using 2014-01.

In [112]:
size='small'
rides = datautils.read_rides(f'data/yellow_tripdata_2014-01_{size}.csv')
rides = datautils.clean_rides(rides)
rides['join_datetime'] = pd.to_datetime(rides.pickup_datetime.dt.strftime("%Y-%m-%d %H"))
rides.head()

,pickup_datetime,pickup_longitude,pickup_latitude,join_datetime
0,2014-01-25 01:53:00,-73.979040,40.761767,2014-01-25 01:00:00
1,2014-01-10 22:11:20,-73.776607,40.645137,2014-01-10 22:00:00
2,2014-01-14 19:27:05,-73.981940,40.761639,2014-01-14 19:00:00
3,2014-01-28 13:35:00,-73.970580,40.755682,2014-01-28 13:00:00
4,2014-01-04 21:10:00,-73.983027,40.766362,2014-01-04 21:00:00


In [113]:
# minutes and seconds information are unnecessary.
rides = datautils.add_grid_cols(rides).drop(['pickup_latitude', 'pickup_longitude', 'pickup_datetime'], axis=1)
rides.head()

,join_datetime,grid_x,grid_y
0,2014-01-25 01:00:00,30,47
1,2014-01-10 22:00:00,117,70
2,2014-01-14 19:00:00,29,47
3,2014-01-28 13:00:00,34,48
4,2014-01-04 21:00:00,28,46


In [114]:
#counts = rides.set_index('join_datetime').groupby([pd.Grouper(freq='1H', level='join_datetime'), 'grid_x', 'grid_y']).size()
counts = rides.groupby(['join_datetime', 'grid_x', 'grid_y']).size()
counts = counts.reset_index(name='count')
counts.head() # For an unknown reason, the hour info is implicit in the printed table

,join_datetime,grid_x,grid_y,count
0,2014-01-01,2,53,1
1,2014-01-01,4,51,1
2,2014-01-01,14,58,4
3,2014-01-01,14,59,1
4,2014-01-01,15,56,1


In [115]:
metar = datautils.read_metar('data/metar_data/lga_2014-01.csv')
metar.head()

,datetime,precip_in,fahrenheit
0,2013-12-31 23:00:00,0.0,26.96
1,2014-01-01 00:00:00,0.0,26.06
2,2014-01-01 01:00:00,0.0,24.98
3,2014-01-01 02:00:00,0.0,24.98
4,2014-01-01 03:00:00,0.0,24.98


In [116]:
# since there are hours without weather info, some rides are dropped.
df = pd.merge(counts, metar, left_on='join_datetime', right_on='datetime', how='inner')
df['weekday'] = df.datetime.dt.weekday
df['hour'] = df.datetime.dt.hour
df = df.drop(['join_datetime', 'datetime'], axis=1)
# data ready for model training.
df.head()

,grid_x,grid_y,count,precip_in,fahrenheit,weekday,hour
0,2,53,1,0.0,26.06,2,0
1,4,51,1,0.0,26.06,2,0
2,14,58,4,0.0,26.06,2,0
3,14,59,1,0.0,26.06,2,0
4,15,56,1,0.0,26.06,2,0


In [117]:
# 9866 - 9452 = 414 rides were dropped. For each missing hour, there are several corresponding grid cells.
df_left_join = pd.merge(rides, metar, left_on='join_datetime', right_on='datetime', how='left')
df_left_join.shape, df.shape

((493141, 6), (174027, 7))

In [118]:
features = ['weekday', 'hour', 'grid_x', 'grid_y', 'fahrenheit', 'precip_in']
X = df[features].values
y = df['count'].values

In [119]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=1)
X_train, X_dev, y_train, y_dev = train_test_split(X_train, y_train,test_size=0.1, random_state=2)

In [120]:
len(X_train), len(X_dev), len(X_test)

(140961, 15663, 17403)

In [121]:
rf = RandomForestRegressor()
rf.fit(X_train, y_train)
rf.score(X_dev, y_dev)

0.4664394125855953

In [122]:
y_pred = rf.predict(X_dev)
np.sum(np.square(y_dev - y_pred))

52977.675681816894

In [124]:
dfdf = pd.DataFrame({'true': y_dev, 'pred': y_pred})
dfdf[dfdf['true'] > 20]

,true,pred
738,26,10.40
2508,21,12.60
4832,24,15.20
5250,28,15.71
9326,21,16.41
12945,27,21.60
13026,22,10.90
13580,21,15.20


## Develop read_metar

In [28]:
usecols = ['valid', 'tmpf', ' p01i'] 
metar = pd.read_csv('data/metar_data/lga_2014-01.csv', usecols=usecols)
metar.columns = ['datetime', 'fahrenheit', 'precip_in']
metar['datetime'] = pd.to_datetime(metar['datetime'])
metar.head() # the last record 

,datetime,fahrenheit,precip_in
0,2013-12-31 23:51:00,26.96,0.0
1,2014-01-01 00:51:00,26.06,0.0
2,2014-01-01 01:51:00,24.98,0.0
3,2014-01-01 02:51:00,24.98,0.0
4,2014-01-01 03:51:00,24.98,0.0


In [26]:
fifth = metar[metar.datetime.dt.day==5]
fifth

,datetime,fahrenheit,precip_in
118,2014-01-05 00:51:00,26.96,0.00
119,2014-01-05 01:51:00,26.96,0.00
120,2014-01-05 02:51:00,26.06,0.00
121,2014-01-05 03:51:00,26.96,0.00
122,2014-01-05 04:51:00,26.06,0.00
123,2014-01-05 05:51:00,26.96,0.00
124,2014-01-05 06:51:00,28.94,0.00
125,2014-01-05 07:51:00,30.02,0.00
126,2014-01-05 08:05:00,30.20,0.00
127,2014-01-05 08:51:00,30.92,0.00


In [27]:
sixth = metar[metar.datetime.dt.day == 6]
sixth.head()

,datetime,fahrenheit,precip_in
165,2014-01-06 00:15:00,50.0,0.00
166,2014-01-06 00:34:00,50.0,0.00
167,2014-01-06 00:47:00,50.0,0.00
168,2014-01-06 00:51:00,50.0,0.03
169,2014-01-06 01:00:00,50.0,0.01


In [24]:
last_day_of_month = metar[metar.datetime.dt.day==31]
last_day_of_month

,datetime,fahrenheit,precip_in
0,2013-12-31 23:51:00,26.96,0.0
905,2014-01-31 00:21:00,26.60,0.0
906,2014-01-31 00:51:00,26.06,0.0
907,2014-01-31 01:11:00,26.60,0.0
908,2014-01-31 01:51:00,26.06,0.0
909,2014-01-31 02:51:00,26.06,0.0
910,2014-01-31 03:51:00,28.04,0.0
911,2014-01-31 04:51:00,28.94,0.0
912,2014-01-31 05:51:00,28.94,0.0
913,2014-01-31 06:51:00,28.94,0.0


In [20]:
precip = metar[['datetime','precip_in']]
precip['month_day'] = precip.datetime.dt.strftime("%m/%d")
precip.groupby('month_day').size()
precip.groupby('month_day').apply(lambda grp: sum(grp.datetime.dt.minute == 51))
# 2nd, 5th, and 31st lost one record.

month_day
01/01    24
01/02    23
01/03    24
01/04    24
01/05    23
01/06    24
01/07    24
01/08    24
01/09    24
01/10    24
01/11    24
01/12    24
01/13    24
01/14    24
01/15    24
01/16    24
01/17    24
01/18    24
01/19    24
01/20    24
01/21    24
01/22    24
01/23    24
01/24    24
01/25    24
01/26    24
01/27    24
01/28    24
01/29    24
01/30    24
01/31    23
12/31     1
dtype: int64

In [7]:
metar[metar.datetime.dt.day==2]

,datetime,precip_in,fahrenheit,month_day
25,2014-01-02 00:00:00,0.00,32.00,01/02
26,2014-01-02 01:00:00,0.00,32.00,01/02
27,2014-01-02 02:00:00,0.00,32.00,01/02
28,2014-01-02 03:00:00,0.00,30.02,01/02
29,2014-01-02 04:00:00,0.00,28.22,01/02
30,2014-01-02 05:00:00,0.01,26.96,01/02
31,2014-01-02 06:00:00,0.00,26.96,01/02
32,2014-01-02 07:00:00,0.00,26.96,01/02
33,2014-01-02 08:00:00,0.00,26.96,01/02
34,2014-01-02 09:00:00,0.00,26.96,01/02


In [6]:
metar['month_day'] = metar.datetime.dt.strftime("%m/%d")
metar.groupby('month_day').size()

month_day
01/01    24
01/02    23
01/03    24
01/04    24
01/05    23
01/06    24
01/07    24
01/08    24
01/09    24
01/10    24
01/11    24
01/12    24
01/13    24
01/14    24
01/15    24
01/16    24
01/17    24
01/18    24
01/19    24
01/20    24
01/21    24
01/22    24
01/23    24
01/24    24
01/25    24
01/26    24
01/27    24
01/28    24
01/29    24
01/30    24
01/31    23
12/31     1
dtype: int64